# Dependencies

In [75]:
import pandas as pd
import sqlite3 as sql
import numpy as np
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score


# Getting Data

In [76]:
stats_cnx = sql.connect('../../Data/Combined Clean Historical Data/Historical_Fantasy_Stats.db')
stats = pd.read_sql_query('SELECT * FROM Historical_Fantasy_Points ', stats_cnx)
stats_cnx.close()

In [77]:
stats['season'] = pd.to_datetime(stats['season'], format='%Y')

In [78]:
stats_prep = stats.copy()

# Manipulating Data

In [79]:
stats_prep['season'] = pd.to_datetime(stats_prep['season'], format='%Y')
stats_prep['nfanpts'] = stats_prep.groupby(['Name','player_id']).shift(-1)['fantast_pts']
stats_prep = stats_prep[stats_prep['nfanpts'].notna()]


In [80]:
# This is to take the players who record more then 40 fantasy points in a season.
stats_prep = stats_prep[stats_prep['nfanpts'] >= 40]
stats_prep

,Name,Position,player_id,season,fantast_pts,completions,attempts,passing_yards,passing_tds,interceptions,...,yac_sh,wopr,ry_sh,rtd_sh,rfd_sh,rtdfd_sh,dom,w8dom,yptmpa,nfanpts
3,Donald Driver,WR,4541,2010-01-01,129.50,0,0,0,0,0,...,0.093617,0.391267,0.138957,0.142857,0.153439,0.152074,0.140907,0.139737,1.116601,117.50
6,Tony Gonzalez,TE,6101,2010-01-01,171.60,0,0,0,0,0,...,0.139824,0.421116,0.176107,0.214286,0.195000,0.197368,0.195197,0.183743,1.136915,209.50
7,Tony Gonzalez,TE,6101,2011-01-01,209.50,0,0,0,0,0,...,0.136058,0.448635,0.200458,0.241379,0.246512,0.245902,0.220919,0.208642,1.473064,234.00
8,Tony Gonzalez,TE,6101,2012-01-01,234.00,0,0,0,0,0,...,0.122271,0.456716,0.197076,0.250000,0.264228,0.262590,0.223538,0.207661,1.512195,218.90
10,Matt Hasselbeck,QB,7091,2010-01-01,148.04,266,444,3001,12,17,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,190.04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5371,Josh Jacobs,RB,35700,2019-01-01,191.60,0,0,0,0,0,...,0.110507,0.088289,0.052432,0.000000,0.051282,0.045977,0.026216,0.041946,0.401937,231.30
5373,Miles Boykin,WR,35703,2019-01-01,50.80,0,0,0,0,0,...,0.010118,0.196293,0.073442,0.103448,0.072993,0.078313,0.088445,0.079443,0.578947,69.60
5375,Drew Lock,QB,35704,2019-01-01,68.00,100,156,1020,7,3,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,179.32
5377,Daniel Jones,QB,35710,2019-01-01,210.98,284,459,3027,24,12,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,178.02


In [81]:
stats_prep.rename(columns = {'nfanpts':'target'}, inplace = True)

In [82]:
unwanted_col = ['Name', 'Position', 'player_id','fantast_pts','rushing_epa', 
           'receiving_2pt_conversions', 'special_teams_tds', 'rushing_2pt_conversions',
           'rushing_fumbles_lost']

stats_prep.drop(unwanted_col, axis=1, inplace=True)
predictors = list(stats_prep)
unwanted_predictor = {'season', 'target'}
predictors = [e for e in predictors if e not in unwanted_predictor]

In [83]:
stats_prep = stats_prep.set_index(['season'])
stats_prep = stats_prep.sort_index()

In [84]:
stats_prep.apply(pd.isnull).sum()

completions                    0
attempts                       0
passing_yards                  0
passing_tds                    0
interceptions                  0
sacks                          0
sack_yards                     0
sack_fumbles                   0
sack_fumbles_lost              0
passing_air_yards              0
passing_yards_after_catch      0
passing_first_downs            0
passing_epa                    0
passing_2pt_conversions        0
carries                        0
rushing_yards                  0
rushing_tds                    0
rushing_fumbles                0
rushing_first_downs            0
receptions                     0
targets                        0
receiving_yards                0
receiving_tds                  0
receiving_fumbles              0
receiving_fumbles_lost         0
receiving_air_yards            0
receiving_yards_after_catch    0
receiving_first_downs          0
receiving_epa                  0
games                          0
tgt_sh    

In [85]:
stats_prep['rtd_sh'] = stats_prep['rtd_sh'].fillna(stats_prep['rtd_sh'].mean())
stats_prep['dom'] = stats_prep['dom'].fillna(stats_prep['dom'].mean())
stats_prep['w8dom'] = stats_prep['w8dom'].fillna(stats_prep['w8dom'].mean())

In [86]:
stats_prep.apply(pd.isnull).sum()

completions                    0
attempts                       0
passing_yards                  0
passing_tds                    0
interceptions                  0
sacks                          0
sack_yards                     0
sack_fumbles                   0
sack_fumbles_lost              0
passing_air_yards              0
passing_yards_after_catch      0
passing_first_downs            0
passing_epa                    0
passing_2pt_conversions        0
carries                        0
rushing_yards                  0
rushing_tds                    0
rushing_fumbles                0
rushing_first_downs            0
receptions                     0
targets                        0
receiving_yards                0
receiving_tds                  0
receiving_fumbles              0
receiving_fumbles_lost         0
receiving_air_yards            0
receiving_yards_after_catch    0
receiving_first_downs          0
receiving_epa                  0
games                          0
tgt_sh    

In [87]:
train = stats_prep.loc[:'2018-01-01']
test = stats_prep.loc['2019-01-01':]

In [88]:
train

,completions,attempts,passing_yards,passing_tds,interceptions,sacks,sack_yards,sack_fumbles,sack_fumbles_lost,passing_air_yards,...,yac_sh,wopr,ry_sh,rtd_sh,rfd_sh,rtdfd_sh,dom,w8dom,yptmpa,target
season,,,,,,,,,,,,,,,,,,,,,
2010-01-01,0,0,0,0,0,0,0,0,0,0,...,0.093617,0.391267,0.138957,0.142857,0.153439,0.152074,0.140907,0.139737,1.116601,117.50
2010-01-01,40,66,433,3,2,7,38,1,1,585,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,44.12
2010-01-01,0,0,0,0,0,0,0,0,0,0,...,0.158786,0.372996,0.152019,0.095238,0.160000,0.152047,0.123629,0.140663,1.273632,124.70
2010-01-01,0,0,0,0,0,0,0,0,0,0,...,0.287381,0.664643,0.318303,0.416667,0.367470,0.373684,0.367485,0.337976,2.067437,218.40
2010-01-01,0,0,0,0,0,0,0,0,0,0,...,0.030162,0.211278,0.046948,0.200000,0.090909,0.102041,0.123474,0.077559,0.341880,62.10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-01-01,0,0,0,0,0,0,0,0,0,0,...,0.140141,0.263659,0.111571,0.051282,0.126126,0.114943,0.081427,0.099513,0.819257,47.70
2018-01-01,0,0,0,0,0,0,0,0,0,0,...,0.072569,0.282122,0.103831,0.130435,0.127820,0.128205,0.117133,0.109152,0.757180,100.80
2018-01-01,0,0,0,0,0,0,0,0,0,0,...,0.195709,0.184480,0.100681,0.103448,0.132275,0.128440,0.102064,0.101234,0.747387,154.00


In [89]:
test

,completions,attempts,passing_yards,passing_tds,interceptions,sacks,sack_yards,sack_fumbles,sack_fumbles_lost,passing_air_yards,...,yac_sh,wopr,ry_sh,rtd_sh,rfd_sh,rtdfd_sh,dom,w8dom,yptmpa,target
season,,,,,,,,,,,,,,,,,,,,,
2019-01-01,0,0,0,0,0,0,0,0,0,0,...,0.034672,0.060158,0.010157,0.000000,0.000000,0.000000,0.005078,0.008126,0.085938,65.3
2019-01-01,0,0,0,0,0,0,0,0,0,0,...,0.094042,0.362897,0.176015,0.150000,0.181818,0.177632,0.163007,0.170812,1.213740,97.8
2019-01-01,0,0,0,0,0,0,0,0,0,0,...,0.110856,0.550590,0.226256,0.263158,0.216049,0.220994,0.244707,0.233637,1.495202,224.1
2019-01-01,303,489,3322,17,16,46,397,10,6,4001,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,45.0
2019-01-01,0,0,0,0,0,0,0,0,0,0,...,0.076923,0.707359,0.212121,0.500000,0.230769,0.266667,0.356061,0.269697,2.000000,117.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-01-01,0,0,0,0,0,0,0,0,0,0,...,0.062914,0.100712,0.052408,0.000000,0.042254,0.038961,0.026204,0.041926,0.300813,45.0
2019-01-01,0,0,0,0,0,0,0,0,0,0,...,0.182447,0.457605,0.211746,0.200000,0.227273,0.224490,0.205873,0.209397,1.451264,100.9
2019-01-01,0,1,0,0,0,0,0,0,0,12,...,0.168809,0.645740,0.313628,0.230769,0.248485,0.246073,0.272198,0.297056,2.539326,328.6


# Modelling 

In [90]:
reg = Ridge(alpha=3030.304)

In [91]:
reg.fit(train[predictors], train["target"])

Ridge(alpha=3030.304)

In [92]:
predictions = reg.predict(test[predictors])

In [93]:
mean_squared_error(test["target"], predictions)

4326.490667676984

In [94]:
r2_score(test["target"], predictions)

0.37729517562516923

In [95]:
combined = pd.concat([test["target"], pd.Series(predictions, index=test.index)], axis=1)
combined.columns = ["actual", "predictions"]

In [96]:
combined

,actual,predictions
season,,
2019-01-01,65.3,97.179888
2019-01-01,97.8,137.155957
2019-01-01,224.1,151.654428
2019-01-01,45.0,196.365355
2019-01-01,117.1,106.540399
...,...,...
2019-01-01,45.0,86.288132
2019-01-01,100.9,155.183677
2019-01-01,328.6,189.548940


In [97]:
features = stats_prep.columns.tolist()
features = features[:-1]
model_coeff = pd.DataFrame({"Feature":features,"Coefficients":reg.coef_})
model_coeff


,Feature,Coefficients
0,completions,0.342999
1,attempts,-0.186019
2,passing_yards,0.025675
3,passing_tds,1.044500
4,interceptions,0.705944
5,sacks,0.231069
6,sack_yards,-0.012907
7,sack_fumbles,0.466887
8,sack_fumbles_lost,-0.055171
9,passing_air_yards,0.004037


In [98]:
corr = stats_prep.corr()["target"]
sorted_corr = corr.sort_values(kind="quicksort")
sorted_corr[0:6]

receiving_fumbles_lost    0.074289
receiving_fumbles         0.102789
rtd_sh                    0.116292
ay_sh                     0.122301
wopr                      0.141068
tgt_sh                    0.147185
Name: target, dtype: float64

In [99]:
import pickle

In [100]:
filename = "ridge_model_low_alpha.pkl"
pickle.dump(reg, open(filename, 'wb'))


# load the model from disk
#loaded_model = pickle.load(open(filename, 'rb'))
#result = loaded_model.score(X_test, Y_test)

In [101]:
combined['Differences'] = combined['actual'] - combined['predictions']

In [102]:
combined.describe()

,actual,predictions,Differences
count,256.000000,256.000000,256.000000
mean,144.777969,145.312261,-0.534292
std,83.517342,51.349383,65.902737
min,40.500000,78.874301,-237.078895
25%,80.025000,102.176301,-45.056812
50%,127.800000,133.836009,-6.015134
75%,188.525000,183.015059,39.700686
max,396.060000,327.478895,169.941257


In [103]:
combined.agg(['mean', 'median', 'std', 'var', 'skew', 'kurt'])

,actual,predictions,Differences
mean,144.777969,145.312261,-0.534292
median,127.800000,133.836009,-6.015134
std,83.517342,51.349383,65.902737
var,6975.146374,2636.759131,4343.170710
skew,0.996094,0.894616,0.094432
kurt,0.410660,0.196622,0.712820


In [104]:
rmse = r2_score(test["target"], predictions)
rmse

0.37729517562516923

In [105]:
stats_prep.corr(method= 'kendall')["target"]

completions                    0.211520
attempts                       0.213708
passing_yards                  0.211144
passing_tds                    0.220358
interceptions                  0.219833
sacks                          0.210213
sack_yards                     0.211823
sack_fumbles                   0.216977
sack_fumbles_lost              0.202600
passing_air_yards              0.214560
passing_yards_after_catch      0.213192
passing_first_downs            0.211170
passing_epa                    0.127831
passing_2pt_conversions        0.179399
carries                        0.132998
rushing_yards                  0.126081
rushing_tds                    0.134140
rushing_fumbles                0.177988
rushing_first_downs            0.137980
receptions                     0.138791
targets                        0.125511
receiving_yards                0.126066
receiving_tds                  0.118153
receiving_fumbles              0.088119
receiving_fumbles_lost         0.069726


# 2022 Predictions

In [106]:
stats_cnx = sql.connect('../../Data/Yearly Data as Database/Stats_with_names_2021.db')
stats_2021 = pd.read_sql_query('SELECT * FROM Stats_with_names_2021 ', stats_cnx)
stats_cnx.close()

In [107]:
stats_2021['wopr'] = stats_2021['wopr_x'] + stats_2021['wopr_y']

In [108]:
stats_2021.columns

Index(['Name', 'Position', 'player_id', 'season', 'season_type', 'completions',
       'attempts', 'passing_yards', 'passing_tds', 'interceptions', 'sacks',
       'sack_yards', 'sack_fumbles', 'sack_fumbles_lost', 'passing_air_yards',
       'passing_yards_after_catch', 'passing_first_downs', 'passing_epa',
       'passing_2pt_conversions', 'pacr', 'carries', 'rushing_yards',
       'rushing_tds', 'rushing_fumbles', 'rushing_fumbles_lost',
       'rushing_first_downs', 'rushing_epa', 'rushing_2pt_conversions',
       'receptions', 'targets', 'receiving_yards', 'receiving_tds',
       'receiving_fumbles', 'receiving_fumbles_lost', 'receiving_air_yards',
       'receiving_yards_after_catch', 'receiving_first_downs', 'receiving_epa',
       'receiving_2pt_conversions', 'racr', 'target_share', 'air_yards_share',
       'wopr_x', 'special_teams_tds', 'games', 'tgt_sh', 'ay_sh', 'yac_sh',
       'wopr_y', 'ry_sh', 'rtd_sh', 'rfd_sh', 'rtdfd_sh', 'dom', 'w8dom',
       'yptmpa', 'wopr'],
   

In [109]:
stats_2021['rtd_sh'] = stats_2021['rtd_sh'].fillna(stats_2021['rtd_sh'].mean())
stats_2021['dom'] = stats_2021['dom'].fillna(stats_2021['dom'].mean())
stats_2021['w8dom'] = stats_2021['w8dom'].fillna(stats_2021['w8dom'].mean())

In [110]:
pred_test = reg.predict(stats_2021[predictors])

In [111]:
stats_2021['rob_fpts_2022'] = pred_test

In [112]:
stats_2021

,Name,Position,player_id,season,season_type,completions,attempts,passing_yards,passing_tds,interceptions,...,wopr_y,ry_sh,rtd_sh,rfd_sh,rtdfd_sh,dom,w8dom,yptmpa,wopr,rob_fpts_2022
0,Tom Brady,QB,19596,2021,REG,485,719,5316.0,43,12.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,306.755296
1,Andy Lee,PN,22824,2021,REG,0,0,0.0,0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,96.947096
2,Ben Roethlisberger,QB,22924,2021,REG,390,605,3740.0,22,10.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,200.617049
3,Aaron Rodgers,QB,23459,2021,REG,366,531,4115.0,37,4.0,...,0.001939,-0.000923,0.000000,0.000000,0.000000,-0.000461,-0.000738,-0.007156,0.042417,259.812011
4,Ryan Fitzpatrick,QB,23682,2021,REG,3,6,13.0,0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,97.700403
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
639,Hunter Long,TE,37004,2021,REG,0,0,0.0,0,0.0,...,0.047322,0.010076,0.000000,0.000000,0.000000,0.005038,0.008060,0.068966,0.197952,89.379751
640,Tommy Tremble,TE,37005,2021,REG,0,0,0.0,0,0.0,...,0.165644,0.063003,0.090909,0.074627,0.075862,0.076956,0.068584,0.360000,2.579061,82.999354
641,Tre' McKitty,TE,37011,2021,REG,0,0,0.0,0,0.0,...,0.054752,0.023974,0.000000,0.030303,0.026786,0.011987,0.019180,0.177165,0.433019,87.684597
642,Trey Lance,QB,37012,2021,REG,41,71,603.0,5,2.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,120.062240


In [297]:
cols = list(stats_2021)
cols.insert(2, cols.pop(cols.index('rob_fpts_2022')))
stats_2021 = stats_2021.loc[:, cols]
stats_2021

,Name,Position,rob_fpts_2022,player_id,season,season_type,completions,attempts,passing_yards,passing_tds,...,yac_sh,wopr_y,ry_sh,rtd_sh,rfd_sh,rtdfd_sh,dom,w8dom,yptmpa,wopr
0,Tom Brady,QB,311.423427,19596,2021,REG,485,719,5316.0,43,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,Andy Lee,PN,31.532519,22824,2021,REG,0,0,0.0,0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,Ben Roethlisberger,QB,157.575316,22924,2021,REG,390,605,3740.0,22,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,Aaron Rodgers,QB,249.166523,23459,2021,REG,366,531,4115.0,37,...,0.000000,0.001939,-0.000923,0.000000,0.000000,0.000000,-0.000461,-0.000738,-0.007156,0.042417
4,Ryan Fitzpatrick,QB,31.234062,23682,2021,REG,3,6,13.0,0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
639,Hunter Long,TE,27.745739,37004,2021,REG,0,0,0.0,0,...,0.015625,0.047322,0.010076,0.000000,0.000000,0.000000,0.005038,0.008060,0.068966,0.197952
640,Tommy Tremble,TE,54.562206,37005,2021,REG,0,0,0.0,0,...,0.048985,0.165644,0.063003,0.090909,0.074627,0.075862,0.076956,0.068584,0.360000,2.579061
641,Tre' McKitty,TE,34.541276,37011,2021,REG,0,0,0.0,0,...,0.036836,0.054752,0.023974,0.000000,0.030303,0.026786,0.011987,0.019180,0.177165,0.433019
642,Trey Lance,QB,72.691525,37012,2021,REG,41,71,603.0,5,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [113]:
stats_2021['rob_fpts_2022'] = stats_2021['rob_fpts_2022'].apply(lambda x: round(x, 2))
stats_2021

,Name,Position,player_id,season,season_type,completions,attempts,passing_yards,passing_tds,interceptions,...,wopr_y,ry_sh,rtd_sh,rfd_sh,rtdfd_sh,dom,w8dom,yptmpa,wopr,rob_fpts_2022
0,Tom Brady,QB,19596,2021,REG,485,719,5316.0,43,12.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,306.76
1,Andy Lee,PN,22824,2021,REG,0,0,0.0,0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,96.95
2,Ben Roethlisberger,QB,22924,2021,REG,390,605,3740.0,22,10.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,200.62
3,Aaron Rodgers,QB,23459,2021,REG,366,531,4115.0,37,4.0,...,0.001939,-0.000923,0.000000,0.000000,0.000000,-0.000461,-0.000738,-0.007156,0.042417,259.81
4,Ryan Fitzpatrick,QB,23682,2021,REG,3,6,13.0,0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,97.70
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
639,Hunter Long,TE,37004,2021,REG,0,0,0.0,0,0.0,...,0.047322,0.010076,0.000000,0.000000,0.000000,0.005038,0.008060,0.068966,0.197952,89.38
640,Tommy Tremble,TE,37005,2021,REG,0,0,0.0,0,0.0,...,0.165644,0.063003,0.090909,0.074627,0.075862,0.076956,0.068584,0.360000,2.579061,83.00
641,Tre' McKitty,TE,37011,2021,REG,0,0,0.0,0,0.0,...,0.054752,0.023974,0.000000,0.030303,0.026786,0.011987,0.019180,0.177165,0.433019,87.68
642,Trey Lance,QB,37012,2021,REG,41,71,603.0,5,2.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,120.06


In [114]:
predicted_fpts_2022 = stats_2021.filter(['Name','Position','rob_fpts_2022'], axis=1)

In [115]:
predicted_fpts_2022.sort_values(by=['rob_fpts_2022'], ascending=False)

,Name,Position,rob_fpts_2022
265,Cooper Kupp,WR,307.81
363,Josh Allen,QB,307.73
0,Tom Brady,QB,306.76
255,Patrick Mahomes,QB,305.61
538,Justin Herbert,QB,304.61
...,...,...,...
49,Josh Gordon,WR,75.08
351,Kalen Ballage,RB,73.82
174,C.J. Ham,RB,73.21
308,Ian Thomas,TE,71.66


In [119]:
cnx = sql.connect("Predicted_fpts_2022_Alpha_3030.db") #input Database Name
cursor = cnx.cursor() # Creates Cursor if needed comment out

In [120]:
predicted_fpts_2022.to_sql('Predicted_fpts_2022_Alpha_3030', cnx, if_exists='replace', index = False) # Exports data to an SQL Lite Database

644

In [121]:
cursor.close()
cnx.close()   